In [6]:
import tkp.db  # NOTE: this only works with the queryqueue branch of TraP
import logging

from tkp.db.transient import transients
from pandas import DataFrame

logging.basicConfig(level=logging.INFO)

### Database connection settings

In [2]:
engine = 'postgresql'
host = 'localhost'
port = 5433
user = 'gijs'
password = user
database = user
query_loglevel = logging.WARNING  # Set to INFO to see queries, otherwise WARNING

### connect to the database

In [3]:
logging.getLogger('sqlalchemy.engine').setLevel(query_loglevel)
db = tkp.db.Database(engine=engine, host=host, port=port,
                     user=user, password=password, database=database)
db.connect()
session = db.Session()

### Execute the query

In [4]:
transient_query = transients(session, dataset=1)

In [5]:
columns = [c['name'] for c in transient_query.column_descriptions]
df = DataFrame(transient_query.all(), columns=columns)
df

,id,wm_ra,wm_decl,wm_uncertainty_ew,wm_uncertainty_ns,xtrsrc,dataset,datapoints,v_int,eta_int,band,newsource,sigma_rms_max,sigma_rms_min,lightcurve_max,lightcurve_avg,lightcurve_median
0,7,141.551613,51.726530,0.001914,0.001914,7,1,4,0,0,23,None,None,None,0.902167,0.795168,0.788349
1,6,141.697030,52.094119,0.001418,0.001418,6,1,4,0,0,23,None,None,None,4.193544,3.675341,3.627884
2,5,142.095178,53.272211,0.001410,0.001410,5,1,4,0,0,23,None,None,None,6.507488,5.517400,5.441342
3,1,144.348609,52.399574,0.001788,0.001788,1,1,4,0,0,25,None,None,None,1.211168,0.958419,0.915310
4,1,144.348609,52.399574,0.001788,0.001788,1,1,4,0,0,26,None,None,None,1.211168,0.958419,0.915310
5,2,143.874411,53.499957,0.001864,0.001864,2,1,4,0,0,26,None,None,None,1.207494,1.058260,1.092771
6,2,143.874411,53.499957,0.001864,0.001864,2,1,4,0,0,25,None,None,None,1.207494,1.058260,1.092771
7,13,141.326463,52.640982,0.003015,0.003015,20,1,2,0,0,24,None,None,None,0.517477,0.485289,0.485289
8,4,142.216853,53.877425,0.001698,0.001698,4,1,4,0,0,26,None,None,None,1.066289,1.001739,0.999654
9,3,142.448058,51.946918,0.001888,0.001888,3,1,4,0,0,24,None,None,None,1.105070,1.012073,1.027755
